In [1]:
import pandas as pd
import music_model as music
import copy

In [4]:
music_df = pd.read_csv("music_genre.csv")

In [5]:
#cleaning missing tempo values
for i in range(len(music_df["tempo"])):
    if music_df["tempo"][i] == "?":
        music_df.loc[i,"tempo"] = 0
#making tempo numeric rather than strings
music_df["tempo"] = pd.to_numeric(music_df["tempo"])

#cleaning missing duration values
for i in range(len(music_df["duration_ms"])):
    if music_df["duration_ms"][i] == -1.0:
        music_df.loc[i, "duration_ms"] = 0

In [6]:
#creating a new dataframe with only the genres I am interested in
rap_df = music_df[music_df["music_genre"] == "Rap"]
hiphop_df = music_df[music_df["music_genre"] == "Hip-Hop"]
rap_hiphop_df = pd.concat([rap_df, hiphop_df])

In [7]:
#manually made list of instance_ids for songs I like
liked_rap_hiphop_ids = [79352.0, 67119.0, 36075.0, 38845.0, 38633.0, 55040.0, 61160.0, 49931.0, 90720.0, 75591.0, 31572.0, 79330.0, 91432.0, 36058.0, 23264.0, 28154.0, 25937.0, 34966.0, 24384.0, 44286.0,86885.0, 74562.0,74806.0, 88024.0, 42558.0, 62664.0, 63964.0, 63243.0, 69718.0, 54906.0, 64939.0, 46619.0, 51675.0, 23497.0, 89355.0]

In [8]:
rap_hiphop_df = rap_hiphop_df.set_index("instance_id")

In [9]:
#manually made list of instance_ids for songs I DONT like
bad_ids = [25193.0, 43557.0, 57944.0, 58878.0, 61813.0, 69366.0, 46274.0, 83672.0, 47716.0, 66463.0] 
#creating a train dataframe for song ids and whether or not I like them
train_ids = liked_rap_hiphop_ids+bad_ids
train_y = pd.DataFrame(index = train_ids)
train_y['y'] = False
for i in liked_rap_hiphop_ids:
    train_y[i:i] = True

In [10]:
#creating a train dataframe of all the songs I have chosen before
cols = list(music_df.columns)
cols.remove('instance_id')
train_x = pd.DataFrame(index = train_ids, columns = cols)
for i in train_ids:
    train_x.loc[i] = rap_hiphop_df.loc[i]

In [27]:
#
from music_model import UserPredictor
model = UserPredictor()
model.fit(train_x, train_y)
results = rap_hiphop_df[model.predict(rap_hiphop_df)]
proba = model.predict_proba(rap_hiphop_df)
proba_yes = []

for idx in range(len(proba)):
    proba_yes.append(proba[idx][0])
rap_hiphop_df["proba"] = proba_yes



In [28]:
rap_hiphop_df.sort_values("proba", ascending = False).head(30)

,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre,proba
instance_id,,,,,,,,,,,,,,,,,,
43559.0,Gucci Mane,1st Day Out Tha Feds,52.0,0.00122,0.870,184400.0,0.427,0.000274,E,0.1140,-6.711,Major,0.0694,138.013,4-Apr,0.6340,Rap,0.999994
41845.0,Gucci Mane,1st Day Out Tha Feds,52.0,0.00122,0.870,184400.0,0.427,0.000274,E,0.1140,-6.711,Major,0.0694,138.013,4-Apr,0.6340,Hip-Hop,0.999981
91528.0,Waka Flocka Flame,"OVA (feat. DJ Whoo Kid, Offset & Ca$h Out)",23.0,0.12100,0.772,269427.0,0.484,0.000000,B,0.1790,-10.103,Major,0.1140,135.956,4-Apr,0.3340,Rap,0.999947
59279.0,Fat Joe,Dirt,14.0,0.04500,0.904,0.0,0.535,0.000000,C#,0.1990,-9.767,Major,0.3600,96.003,4-Apr,0.7200,Rap,0.999920
71804.0,empty_field,Or Nah (feat. Wiz Khalifa and DJ Mustard),56.0,0.05440,0.830,251861.0,0.330,0.000000,E,0.1190,-9.156,Major,0.0485,121.974,4-Apr,0.0744,Rap,0.999899
58122.0,Ty Dolla $ign,Dawsin's Breek (feat. Jeremih),52.0,0.00963,0.907,165200.0,0.709,0.000004,B,0.1060,-3.339,Major,0.0594,129.007,4-Apr,0.4390,Rap,0.999892
53031.0,Timbaland,Give It To Me,54.0,0.17000,0.973,234027.0,0.723,0.000282,G#,0.0863,-4.017,Major,0.0569,110.625,4-Apr,0.7510,Rap,0.999859
44997.0,Three 6 Mafia,Sippin On Some Syrup (feat. UGK (Underground K...,56.0,0.00919,0.944,261973.0,0.839,0.000000,B,0.0921,-4.141,Major,0.1080,136.019,4-Apr,0.8920,Rap,0.999852
62648.0,Kodak Black,Candy Paint (feat. Bun B),49.0,0.01950,0.863,218990.0,0.862,0.000000,G#,0.1050,-4.679,Major,0.0545,120.009,4-Apr,0.7030,Rap,0.999832


In [23]:
results#.sort_values("proba", ascending = False)

,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
instance_id,,,,,,,,,,,,,,,,,
28170.0,Smino,L.M.F.,63.0,0.354000,0.898,0.0,0.677,0.000005,C#,0.4490,-6.907,Major,0.3620,155.020,4-Apr,0.864,Rap
36622.0,Rob Zombie,Superbeast,61.0,0.000136,0.518,0.0,0.975,0.827000,G#,0.4200,-4.425,Major,0.0575,154.059,4-Apr,0.320,Rap
44032.0,YG,Who Do You Love?,67.0,0.028100,0.676,233507.0,0.415,0.000000,A,0.0915,-9.752,Major,0.5080,193.206,4-Apr,0.146,Rap
35196.0,Fabolous,Breathe,53.0,0.340000,0.713,0.0,0.910,0.000000,A#,0.3040,-5.586,Minor,0.2820,83.028,4-Apr,0.510,Rap
62158.0,Falling In Reverse,Raised By Wolves,57.0,0.034600,0.368,204693.0,0.871,0.000000,D,0.0599,-3.395,Minor,0.1060,84.585,4-Apr,0.527,Rap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20269.0,Unknown Mortal Orchestra,So Good at Being in Trouble,57.0,0.036300,0.829,230147.0,0.435,0.878000,C,0.1190,-10.136,Major,0.0515,103.816,4-Apr,0.594,Hip-Hop
54580.0,Tee Grizzley,First Day Out,75.0,0.171000,0.587,254694.0,0.711,0.000000,C#,0.1250,-6.330,Major,0.4200,98.008,4-Apr,0.381,Hip-Hop
64552.0,Logic,Growing Pains III,55.0,0.474000,0.514,246773.0,0.730,0.000000,E,0.5110,-8.491,Minor,0.3870,167.655,4-Apr,0.699,Hip-Hop


In [ ]:
#filters out 4427 songs
len(rap_hiphop_df) - len(results)